In [68]:
import os
import re
import numpy as np
import mne
from mne.io import read_raw_brainvision

def get_rawdata_list(): # get the list of rawdata files with .vhdr extension
    rawdata_dir = 'C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/' # directory where the rawdata is stored
    rawdata_list_total = os.listdir(rawdata_dir) # return all files in the directory without directory itself
    rawdata_list = [os.path.join(rawdata_dir, i) for i in rawdata_list_total if re.search('[0-9].vhdr', i)] # return only files with .vhdr extension
    
    return rawdata_list

def get_montage():
    montage_dir = 'C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/CMA-64_REF.bvef'
    montage = mne.channels.read_custom_montage(montage_dir)
    montage.rename_channels(dict(REF='FCz'))
    return montage

def basic_filtering(raw):
    l_freq = 0.5 # low cut-off frequency
    h_freq = None
    iir_params = mne.filter.construct_iir_filter( 
        {'ftype': 'butter', 'order': 4},
        l_freq, None, raw.info['sfreq'],
        'highpass', return_copy=False, verbose=None
    ) # IIR filter parameters
    raw_filtered = raw.copy().filter(
        l_freq=l_freq, h_freq = h_freq, method='iir', 
        iir_params=iir_params, verbose=None
    ) # bandpass filtering
    raw_filtered = raw_filtered.notch_filter(60) # notch filtering
    
    return raw_filtered

In [69]:

rawdata_list = get_rawdata_list()
montage = get_montage()
raw = read_raw_brainvision(rawdata_list[0], preload=True)
raw.set_channel_types({'EOG':'eog'})
raw.set_montage(montage)
raw_filtered = basic_filtering(raw)

Extracting parameters from C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/tACS_2022_April_SU0001_1.vhdr...
Setting channel info structure...
Reading 0 ... 276699  =      0.000 ...   553.398 secs...



IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.50 Hz: -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.5 Hz



C:\Users\YSB\AppData\Local\Temp\ipykernel_21348\2148696495.py:5: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 3301 samples (6.602 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.9s finished


In [70]:
raw.plot(n_channels=64, scalings={'eeg':100e-6})

Using qt as 2D backend.


Channels marked as bad:
['C5', 'Fp2']


In [12]:

np.unique(raw.get_channel_types())

array(['eeg'], dtype='<U3')

In [89]:
dict(zip(raw.ch_names, montage.ch_names))

{'Fp1': 'GND',
 'Fp2': 'REF',
 'F7': 'Fp1',
 'F3': 'Fz',
 'Fz': 'F3',
 'F4': 'F7',
 'F8': 'FT9',
 'FC5': 'FC5',
 'FC1': 'FC1',
 'FC2': 'C3',
 'FC6': 'T7',
 'T7': 'TP9',
 'C3': 'CP5',
 'Cz': 'CP1',
 'C4': 'Pz',
 'T8': 'P3',
 'EOG': 'P7',
 'CP5': 'O1',
 'CP1': 'Oz',
 'CP2': 'O2',
 'CP6': 'P4',
 'FCz': 'P8',
 'P7': 'TP10',
 'P3': 'CP6',
 'Pz': 'CP2',
 'P4': 'Cz',
 'P8': 'C4',
 'TP9': 'T8',
 'O1': 'FT10',
 'Oz': 'FC6',
 'O2': 'FC2',
 'TP10': 'F4',
 'AF7': 'F8',
 'AF3': 'Fp2',
 'AF4': 'AF7',
 'AF8': 'AF3',
 'F5': 'AFz',
 'F1': 'F1',
 'F2': 'F5',
 'F6': 'FT7',
 'FT9': 'FC3',
 'FT7': 'C1',
 'FC3': 'C5',
 'FC4': 'TP7',
 'FT8': 'CP3',
 'FT10': 'P1',
 'C5': 'P5',
 'C1': 'PO7',
 'C2': 'PO3',
 'C6': 'POz',
 'TP7': 'PO4',
 'CP3': 'PO8',
 'CPz': 'P6',
 'CP4': 'P2',
 'TP8': 'CPz',
 'P5': 'CP4',
 'P1': 'TP8',
 'P2': 'C6',
 'P6': 'C2',
 'PO7': 'FC4',
 'PO3': 'FT8',
 'POz': 'F6',
 'PO4': 'AF8',
 'PO8': 'AF4'}

In [ ]:
if __name__ =='__main__':
    